In [ ]:
# import the necessary libraries

import numpy as np
import pandas as pd
import os
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# unzipping the file if required

path = os.getcwd()
path = os.path.join(path, "all_data.zip")
!unzip {path} -d "/content/"

Archive:  /content/all_data.zip
  inflating: /content/all_data/test_c1000_d100.csv  
  inflating: /content/all_data/test_c1000_d1000.csv  
  inflating: /content/all_data/test_c1000_d5000.csv  
  inflating: /content/all_data/test_c1500_d100.csv  
  inflating: /content/all_data/test_c1500_d1000.csv  
  inflating: /content/all_data/test_c1500_d5000.csv  
  inflating: /content/all_data/test_c1800_d100.csv  
  inflating: /content/all_data/test_c1800_d1000.csv  
  inflating: /content/all_data/test_c1800_d5000.csv  
  inflating: /content/all_data/test_c300_d100.csv  
  inflating: /content/all_data/test_c300_d1000.csv  
  inflating: /content/all_data/test_c300_d5000.csv  
  inflating: /content/all_data/test_c500_d100.csv  
  inflating: /content/all_data/test_c500_d1000.csv  
  inflating: /content/all_data/test_c500_d5000.csv  
  inflating: /content/all_data/train_c1000_d100.csv  
  inflating: /content/all_data/train_c1000_d1000.csv  
  inflating: /content/all_data/train_c1000_d5000.csv  
  inf

In [ ]:
# creating a dataframe to store results
df_results = pd.DataFrame(columns=['DataSet_name', 'Loss', 'Learning_rate', 'N_Estimators', 'Subsample', 'Max_depth', 'Accuracy', 'F1_score'])

In [ ]:
# Defining the Gradient Boosting Classifier

def DT():

  # Getting the features and output 
  XValid = validData.iloc[:, 0:-1].values
  YValid = validData.iloc[:, -1].values
  XTest = testData.iloc[:, 0:-1].values
  YTest = testData.iloc[:, -1].values
  
  # Getting the best parameters
  parameters = {'loss': ['deviance', 'exponential'], 'learning_rate': [0.01, 0.1, 1], 'n_estimators': [1, 100], 'subsample': [0.1, 1], 'max_depth': [1, 3, 10]}
  grid = GridSearchCV(GradientBoostingClassifier(), parameters, n_jobs=10, verbose=True)
  grid.fit(XValid, YValid)

  # Combining training and validation data
  trainX1 = trainData.iloc[:, 0:-1]
  validX1 = validData.iloc[:, 0:-1]
  XData = pd.concat([trainX1, validX1], axis=0, copy=True).values

  trainY1 = trainData.iloc[:, -1]
  validY2 = validData.iloc[:, -1]
  YData = pd.concat([trainY1, validY2], axis=0, copy=True).values

  # Final training and Testing
  tree = GradientBoostingClassifier(**grid.best_params_)
  tree = tree.fit(XData, YData)
  yPredict = tree.predict(XTest)

  return grid.best_params_, metrics.accuracy_score(YTest, yPredict), f1_score(YTest, yPredict) 

In [ ]:
# Getting the data
path = os.getcwd()
path = os.path.join(path, 'all_data')
firstNum = [3, 5, 10, 15, 18]
secNum = [1, 10, 50]

for i in firstNum:
  for j in secNum:
    trainData = pd.read_csv(path + "/train_c" + str(i) + "00_d" + str(j) + "00.csv", header=None)
    validData = pd.read_csv(path + "/valid_c" + str(i) + "00_d" + str(j) + "00.csv", header=None)
    testData = pd.read_csv(path + "/test_c" + str(i) + "00_d" + str(j) + "00.csv", header=None)

    Best_Parameters, Acc, F_Score = DT()
    df_results = df_results.append({'DataSet_name': "c" + str(i) + "00_d" + str(j) + "00", 'Loss': Best_Parameters['loss'], 'Learning_rate': Best_Parameters['learning_rate'], 'N_Estimators': Best_Parameters['n_estimators'], 'Subsample': Best_Parameters['subsample'], 'Max_depth': Best_Parameters['max_depth'], 'Accuracy': Acc, 'F1_score': F_Score}, ignore_index=True)


In [ ]:
df_results

,DataSet_name,Loss,Learning_rate,N_Estimators,Subsample,Max_depth,Accuracy,F1_score
0,c300_d100,exponential,0.10,100,1,3,0.8500,0.854369
1,c300_d1000,deviance,0.10,100,1,3,0.9725,0.973210
2,c300_d5000,exponential,1.00,100,1,3,0.9961,0.996114
3,c500_d100,exponential,1.00,100,1,1,0.8550,0.854271
4,c500_d1000,deviance,0.10,100,1,3,0.9845,0.984646
5,c500_d5000,exponential,1.00,100,1,3,0.9983,0.998303
6,c1000_d100,exponential,1.00,100,1,3,0.9750,0.974874
7,c1000_d1000,deviance,1.00,100,1,3,0.9950,0.995010
8,c1000_d5000,exponential,1.00,100,1,3,0.9997,0.999700
9,c1500_d100,exponential,0.10,100,0.1,1,0.9900,0.990099
